# Japanese Text Analysis

In [1]:
import json
import pandas as pd
import requests

from sudachipy import tokenizer
from sudachipy import dictionary
from sudachipy import config

from io import StringIO
import re
from scipy import stats

import unicodedata

## Example of using sudachi for parsing Japanese text.

Check [github](https://github.com/WorksApplications/Sudachi) page for Japanese morphological analyzer Sudachi. ![Sudachi](images/Sudachi.png)

In [2]:
with open(config.SETTINGFILE, "r", encoding="utf-8") as f:
    settings = json.load(f)
tokenizer_obj = dictionary.Dictionary(settings).create()

In [3]:
config.SETTINGFILE

'/Users/ray/scratch/japanese_text_analysis/src/sudachipy/sudachipy/../resources/sudachi.json'

In [4]:
!cat {config.SETTINGFILE}

{
    "systemDict" : "system_full.dic",
    "characterDefinitionFile" : "char.def",
    "inputTextPlugin" : [
        { "class" : "com.worksap.nlp.sudachi.DefaultInputTextPlugin" }
    ],
    "oovProviderPlugin" : [
        { "class" : "com.worksap.nlp.sudachi.MeCabOovProviderPlugin",
          "charDef" : "char.def",
          "unkDef" : "unk.def" },
        { "class" : "com.worksap.nlp.sudachi.SimpleOovProviderPlugin",
          "oovPOS" : [ "補助記号", "一般", "*", "*", "*", "*" ],
          "leftId" : 5968,
          "rightId" : 5968,
          "cost" : 3857 }
    ],
    "pathRewritePlugin" : [
        { "class" : "com.worksap.nlp.sudachi.JoinNumericPlugin",
          "joinKanjiNumeric" : true },
        { "class" : "com.worksap.nlp.sudachi.JoinKatakanaOovPlugin",
          "oovPOS" : [ "名詞", "普通名詞", "一般", "*", "*", "*" ] }
    ]
}


Multi-granular tokenization
(following results are w/ `system_full.dic`
you may not be able to replicate this particular example w/ `system_core.dic`)

In [5]:
mode = tokenizer.Tokenizer.SplitMode.C
[m.surface() for m in tokenizer_obj.tokenize(mode, "医薬品安全管理責任者")]
# => ['医薬品', '安全', '管理責任者']

['医薬品安全管理責任者']

In [6]:
mode = tokenizer.Tokenizer.SplitMode.B
[m.surface() for m in tokenizer_obj.tokenize(mode, "医薬品安全管理責任者")]
# => ['医薬品', '安全', '管理', '責任者']

['医薬品', '安全', '管理', '責任者']

In [7]:
mode = tokenizer.Tokenizer.SplitMode.A
[m.surface() for m in tokenizer_obj.tokenize(mode, "医薬品安全管理責任者")]
# => ['医薬', '品', '安全', '管理', '責任', '者']

['医薬', '品', '安全', '管理', '責任', '者']

In [8]:
s = '複数粒度の分割結果に基づく日本語単語分散表現'
mode = tokenizer.Tokenizer.SplitMode.C
for t in zip([m.surface() for m in tokenizer_obj.tokenize(mode, s)], [m.reading_form() for m in tokenizer_obj.tokenize(mode, s)]):
    print(t)

('複数', 'フクスウ')
('粒度', 'リュウド')
('の', 'ノ')
('分割', 'ブンカツ')
('結果', 'ケッカ')
('に', 'ニ')
('基づく', 'モトヅク')
('日本語', 'ニホンゴ')
('単語', 'タンゴ')
('分散', 'ブンサン')
('表現', 'ヒョウゲン')


In [9]:
s = '分散表現の構築手法'
mode = tokenizer.Tokenizer.SplitMode.C
for t in zip([m.surface() for m in tokenizer_obj.tokenize(mode, s)], [m.reading_form() for m in tokenizer_obj.tokenize(mode, s)]):
    print(t)

('分散', 'ブンサン')
('表現', 'ヒョウゲン')
('の', 'ノ')
('構築', 'コウチク')
('手法', 'シュホウ')


In [10]:
# Morpheme information

m = tokenizer_obj.tokenize(mode, "食べ")[0]

In [11]:
m.surface() # => '食べ'

'食べ'

In [12]:
m.dictionary_form() # => '食べる'

'食べる'

In [13]:
m.reading_form() # => 'タベ'

'タベ'

In [14]:
def get_reading_form(word):
    m = tokenizer_obj.tokenize(mode, word)[0]
    return m.reading_form()

In [15]:
m.part_of_speech() # => ['動詞', '一般', '*', '*', '下一段-バ行', '連用形-一般']

['動詞', '一般', '*', '*', '下一段-バ行', '連用形-一般']

In [16]:
# Normalization

In [17]:
tokenizer_obj.tokenize(mode, "附属")[0].normalized_form()
# => '付属'

'付属'

In [18]:
tokenizer_obj.tokenize(mode, "SUMMER")[0].normalized_form()
# => 'サマー'

'サマー'

In [19]:
tokenizer_obj.tokenize(mode, "シュミレーション")[0].normalized_form()
# => 'シミュレーション'

'シミュレーション'

## Japanese Text Mining

![Japanese Text Mining](images/japanese_text_mining.jpg)
Check out the [Emory University workshop blog](https://scholarblogs.emory.edu/japanese-text-mining/) on Japanese Text Mining. The example notebook cells below repeat the steps in the [tutorial](http://history.emory.edu/RAVINA/JF_text_mining/Guides/Jtextmining_intro_part1.html) of Mark Ravina using python instead of R.

In [20]:
response = requests.get('http://history.emory.edu/RAVINA/JF_text_mining/Guides/data/meiroku_zasshi.txt')

In [21]:
response.encoding = 'utf-8'

In [22]:
data = [t.split('" "') for t in response.text.split('\n')]

In [23]:
data[0]

['"year', 'issue', 'title', 'author', 'text"']

In [24]:
data = [[d.replace('"', '') for d in row] for row in data]

In [25]:
d = data[1][0].split()
d.extend(data[1][1:])

In [26]:
rows = []
for d in data[1:]:
    row = d[0].split()
    row.extend(d[1:])
    rows.append(row)

In [27]:
df = pd.DataFrame(rows, columns = ['index', 'year', 'issue', 'title', 'author', 'text'])

In [28]:
df = df.drop(df.index[-1])

In [29]:
authors = list(df.author)
no_boxes_per_line = 5
[authors[no_boxes_per_line*m: no_boxes_per_line*m+no_boxes_per_line] for m in range(31)]

[['西周', '西村茂樹', '加藤弘之', '森有礼', '津田真道'],
 ['西周', '森有礼', '西村茂樹', '森有礼', '杉亨二'],
 ['津田真道', '西周', '箕作麟祥', '加藤弘之', '杉亨二'],
 ['西周', '西周', '津田真道', '西周', '杉亨二'],
 ['箕作麟祥', '加藤弘之', '津田真道', '西周', '加藤弘之'],
 ['森有礼', '柴田昌吉', '森有礼', '加藤弘之', '箕作麟祥'],
 ['杉亨二', '津田真道', '清水卯三郎', '津田真道', '森有礼'],
 ['箕作秋坪', '杉亨二', '西周', '津田真道', '津田真道'],
 ['箕作麟祥', '西周', '津田真道', '津田真道', '杉亨二'],
 ['中村正直', '阪谷素', '津田真道', '森有礼', '中村正直'],
 ['阪谷素', '西周', '津田真道', '中村正直', '加藤弘之'],
 ['津田真道', '阪谷素', '西周', '箕作麟祥', '杉亨二'],
 ['津田真道', '森有礼', '中村正直', '阪谷素', '津田真道'],
 ['津田真道', '杉亨二', '中村正直', '西周', '神田孝平'],
 ['津田真道', '西周', '津田真道', '加藤弘之', '杉亨二'],
 ['阪谷素', '西周', '神田孝平', '西周', '神田孝平'],
 ['阪谷素', '杉亨二', '津田真道', '森有礼', '阪谷素'],
 ['阪谷素', '西周', '福沢諭吉', '津田真道', '杉亨二'],
 ['阪谷素', '西周', '津田真道', '阪谷素', '清水卯三郎'],
 ['神田孝平', '西周', '神田孝平', '中村正直', '津田真道'],
 ['杉亨二', '西周', '阪谷素', '津田真道', '福沢諭吉'],
 ['津田真道', '神田孝平', '森有礼', '阪谷素', '阪谷素'],
 ['西村茂樹', '西周', '西村茂樹', '柏原孝章', '森有礼'],
 ['津田真道', '柏原孝章', '中村正直', '加藤弘之', '西村茂樹'],
 ['柏原孝章', '福沢諭吉', '西周', '阪谷素', '森有礼'],
 ['

In [30]:
set(authors)

{'中村正直',
 '加藤弘之',
 '杉亨二',
 '柏原孝章',
 '柴田昌吉',
 '森有礼',
 '津田仙',
 '津田真道',
 '清水卯三郎',
 '神田孝平',
 '福沢諭吉',
 '箕作秋坪',
 '箕作麟祥',
 '西周',
 '西村茂樹',
 '阪谷素'}

In [31]:
mask = df.author == '西周'
df[mask][['index', 'title']].head()

,index,title
0,1,洋字を以て国語を書するの論
5,6,非学者職分論
11,12,駁旧相公議一題
15,16,教門論（一）
16,17,煉火石造の説


## Tale of Genji: Significant Terms and Word Clusters.

![源氏物語歌合](images/200014735/image/200014735_00014.jpg)


In [32]:
# The Tale of the Genji consists of 54 chapters. Each chapter is broken into sections and each section into a list of text blocks.
with open('../data/raw/genji_data.json') as fp:
    genji_data = json.load(fp)

In [33]:
chapter_names = list(genji_data.keys())
print(len(chapter_names))

54


In [34]:
# Section keys for a chapter.
chapter = chapter_names[0]
print(chapter, genji_data[chapter].keys())

桐壷 dict_keys(['in11', 'in12', 'in13', 'in14', 'in15', 'in21', 'in22', 'in23', 'in31', 'in32', 'in33', 'in34', 'in35', 'in36', 'in37', 'in38'])


In [35]:
wikipedia_genji = pd.read_html('https://en.wikipedia.org/wiki/The_Tale_of_Genji', attrs={"class": "wikitable"})[0]

In [36]:
wikipedia_genji

,Chapter,Japanese,Waley,Seidensticker,Tyler,Washburn
0,01,Kiritsubo (桐壺),"""Kiritsubo""","""The Paulownia Court""","""The Paulownia Pavilion""","""The Lady of the Paulownia-Courtyard Chambers"""
1,02,Hahakigi (帚木),"""The Broom-Tree""","""The Broom-Tree""","""The Broom-Tree""","""Broom Cypress"""
2,03,Utsusemi (空蝉),"""Utsusemi""","""The Shell of the Locust""","""The Cicada Shell""","""A Molted Cicada Shell"""
3,04,Yūgao (夕顔),"""Yugao""","""Evening Faces""","""The Twilight Beauty""","""The Lady of the Evening Faces"""
4,05,Wakamurasaki (若紫),"""Murasaki""","""Lavender""","""Young Murasaki""","""Little Purple Gromwell"""
5,06,Suetsumuhana (末摘花),"""The Saffron-Flower""","""The Safflower""","""The Safflower""","""The Safflower"""
6,07,Momiji no Ga (紅葉賀),"""The Festival of Red Leaves""","""An Autumn Excursion""","""Beneath the Autumn Leaves""","""An Imperial Celebration of Autumn Foliages"""
7,08,Hana no En (花宴),"""The Flower Feast""","""The Festival of the Cherry Blossoms""","""Under the Cherry Blossoms""","""A Banquet Celebrating Cherry Blossoms"""
8,09,Aoi (葵),"""Aoi""","""Heartvine""","""Heart-to-Heart""","""Leaves of Wild Ginger"""
9,10,Sakaki (榊),"""The Sacred Tree""","""The Sacred Tree""","""The Green Branch""","""A Branch of Sacred Evergreens"""


In [37]:
chapter_names[:1]

['桐壷']

In [38]:
all_text = ''
mode = tokenizer.Tokenizer.SplitMode.C
text_length = 0
chapter_boundaries = []
for chapter in chapter_names:
    chapter_boundaries.append(text_length)
    for section in genji_data[chapter].keys():
        wordlist = []
        for block in genji_data[chapter][section]:
            wordlist = [m.dictionary_form() for m in tokenizer_obj.tokenize(mode, block)]
            text_length += len(wordlist)
            all_text += ' '.join(wordlist)
            print('\r{}'.format(text_length), end='')

691201

In [39]:
tokens = re.findall('\w+', all_text)

In [40]:
len(tokens)

582814

In [41]:
with open('all_text.txt', 'w') as fp:
    fp.write(all_text)

In [42]:
# sents = !ruby pragmatic_segmenter_test.rb all_text.txt

In [43]:
from src.models import WordLevelStatistics

/Users/ray/miniconda3/envs/japanese_text_analysis/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/ray/miniconda3/envs/japanese_text_analysis/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [44]:
# class WordLevelStatistics():
#     # Copyright 2014 Shubhanshu Mishra. All rights reserved.
#     #
#     # This library is free software; you can redistribute it and/or
#     # modify it under the same terms as Python itself.
#     def __init__(self, word_pos=None, corpus_file=None, percentile_C=95):
#         '''This package is a port of the perl module Algorithm::WordLevelStatistics by
#         Francesco Nidito which can be found at:
#         http://search.cpan.org/~nids/Algorithm-WordLevelStatistics-0.03/

#         The code is an implementation of the spatial statistics described in
#         the following paper:
#         @article{carpena2009level,
#           title={Level statistics of words: Finding keywords in literary texts and symbolic sequences},
#           author={Carpena, P and Bernaola-Galv{\'a}n, P and Hackenberg, M and Coronado, AV and Oliver, JL},
#           journal={Physical Review E},
#           volume={79},
#           number={3},
#           pages={035102},
#           year={2009},
#           publisher={APS}
#         }

#         Author: Shubhanshu Mishra
#         Published: December 29, 2014
#         License: GPL3
#         '''
#         if percentile_C is not None:
#             self.percentile_C = percentile_C

#         if word_pos is not None:
#             self.word_pos = word_pos
#         elif corpus_file is not None:
#             self.word_pos = dict()
#             self.pos_counter = 0
#             if isinstance(corpus_file, list):
#                 for c in corpus_file:
#                     self.gen_word_pos(c)
#             else:
#                 self.gen_word_pos(corpus_file)

#     def gen_word_pos(self, corpus_file):
#         # with open(corpus_file, encoding='utf-8') as fp:
#         text = corpus_file.read()  # .lower()
#         tokens = re.findall('\w+', text)
#         for t in tokens:
#             if t not in self.word_pos:
#                 self.word_pos[t] = []
#             self.word_pos[t].append(self.pos_counter)
#             self.pos_counter += 1

#     def compute_spectra(self):
#         if self.word_pos is None or len(self.word_pos.keys()) < 1:
#             return None
#         # Count total words in the text.
#         self.tot_words = sum([len(self.word_pos[k]) for k in self.word_pos.keys()])

#         # Compute level statistics of all terms
#         self.level_stat = []
#         for k in self.word_pos.keys():
#             ls = self.compute_spectrum(k)
#             self.level_stat.append(ls)

#         # Sort level_stat frequency, use index in this list for vocab.
#         self.level_stat = sorted(self.level_stat,
#                                  key=lambda x: x['count'],
#                                  reverse=True)

#         # Add index to keep track of vocab, higher freq <-> higer index.
#         for n, vocab_entry in enumerate(self.level_stat):
#             vocab_entry['vocab_index'] = n

#         self.threshold = stats.scoreatpercentile(    ## TODO: Compute this directly, dont import extra lib.
#             [t['C'] for t in self.level_stat], self.percentile_C)
#         self.level_stat_thresholded = [t for t in self.level_stat if t['C'] > self.threshold]

#         # Significant terms
#         self.significant_terms = [t['word'] for t in self.level_stat_thresholded]

#     def compute_spectrum(self, word):
#         positions = self.word_pos[word]
#         n = len(positions)
#         ls = {'word': word, 'count': n, 'C': 0, 'sigma_nor': 0}
#         if n > 3:
#             # position -> distance from preceding element in text
#             tmp = [positions[i+1] - positions[i] for i in range(n-1)]
#             # len(tmp) = n-1
#             avg = sum(tmp)*1.0/(n-1)
#             sigma = sum([(k-avg)**2 for k in tmp])*1.0/(n-1)
#             sigma = (sigma**(0.5))/avg

#             p = n*1.0/self.tot_words
#             ls['sigma_nor'] = sigma/((1.0-p)**.5)

#             ls['C'] = (ls['sigma_nor'] - (2.0*n-1.0)/(2.0*n+2.0))\
#                        * ((n**0.5) * (1.0+2.8*n**-0.865))
#         return ls

In [45]:
fp = StringIO(all_text)
word_level_statistics = WordLevelStatistics(corpus_file=fp, percentile_C=98)
word_level_statistics.compute_spectra()

lvls_df = pd.DataFrame(word_level_statistics.level_stat_thresholded)
significant_terms = word_level_statistics.significant_terms
print('Threshold: {}, ({} percentile) find {} significant terms.'.format(
                             word_level_statistics.threshold,
                             word_level_statistics.percentile_C,
                             len(significant_terms)))

Threshold: 6.614132025057848, (98 percentile) find 258 significant terms.


In [46]:
lvls_df = lvls_df.sort_values(by='sigma_nor', ascending=False)
lvls_df['reading form'] = lvls_df.word.map(get_reading_form)
lvls_df.head(100)

,C,count,sigma_nor,vocab_index,word,reading form
107,52.445577,129,5.420635,386,右近,ウコン
205,19.970726,31,4.089600,1326,常陸介,ヒタチノスケ
219,16.028669,24,3.714686,1573,小君,コギミ
129,27.476590,95,3.657707,517,僧都,ソウズ
218,15.713136,25,3.621541,1552,猫,ネコ
208,16.371923,29,3.588781,1376,少納言,ショウナゴン
196,17.070165,37,3.458995,1144,介,スケ
202,15.952786,33,3.400375,1265,斎宮,サイグウ
230,13.077373,19,3.385558,1884,一条,イチジョウ
180,18.556395,50,3.367266,909,効,カイ


In [47]:
block.split(' ')

['',
 '早く早くとお待ちになっていたが、このようにはっきりしないまま帰って来たので、期待が外れて、「かえって遣らないほうがましだった」と、お思いになることがいろいろで、「誰かが隠し置いているのであろうか」と、ご自分の想像の限りを尽くして、放ってお置きになった経験からも、と本にございますようです。']

In [48]:
word = '斎宮'
for n, chapter in enumerate(chapter_names):
    for section in genji_data[chapter].keys():
        for block in genji_data[chapter][section]:
            if word in block:
                print(n+1, chapter, section, block)
                print('='*40)

9 葵 in24 [2-4 斎宮、秋に宮中の初斎院に入る]
9 葵 in24  斎宮は、去年内裏にお入りになるはずであったが、さまざまに差し障ることがあって、この秋にお入りになる。九月には、そのまま野の宮にお移りになる予定なので、二度目の御禊の準備、引き続いて行うはずのところ、まるで妙にぼうっとして、物思いに沈んで悩んでいらっしゃるのを、斎宮寮の官人たち、ひどく重大視して、御祈祷など、あれこれと致す。
10 賢木 in11  斎宮の御下向、近づくなるにつれて、御息所、何となく心細くいらっしゃる。重々しくけむたいご本妻だと思っていらした大殿の姫君もお亡くなりになって後、いくら何でもと世間の人々もお噂申し、宮の内でも期待していたのに、それから後、すっかりお通いがなく、あまりなお仕向けを御覧になると、本当にお嫌いになる事があったのだろうと、すっかりお分かりになってしまったので、一切の未練をお捨てになって、一途にご出立なさる。
10 賢木 in13  斎宮は、幼な心に、決定しなかったご出立が、このように決まってゆくのを、嬉しい、とばかりお思いでいた。世間の人々は、先例のないことだと、非難も同情も、いろいろとお噂申しているようだ。何事でも、人から非難されないような身分の者は気楽なものである。かえって世に抜きん出た方のご身辺は窮屈なことが多いことである。
10 賢木 in14 [1-4 斎宮、宮中へ向かう]
10 賢木 in14  とある。とても取り混んでいる時だが、お返事がある。斎宮のお返事は、女別当にお書かせになっていた。
10 賢木 in14  斎宮のお返事がいかにも成人した詠みぶりなのを、ほほ笑んで御覧になった。「お年の割には、人情がお分かりのようでいらっしゃるな」と、お心が動く。このように普通とは違っためんどうな事には、きっと心動かすご性分なので、「いくらでも拝見しようとすればできたはずであった幼い時を、見ないで過ごしてしまったのは残念なことであった。世の中は無常であるから、お目にかかるようなこともきっとあろう」などと、お思いになる。
10 賢木 in15 [1-5 斎宮、伊勢へ向かう]
10 賢木 in15  斎宮は、十四におなりであった。とてもかわいらしういらっしゃるご様子を、立派に装束をお着せ申されたのが、とても恐いまでに美しくお見えになるのを、帝、お心が動いて

In [49]:
def text_length(text):
    fp = StringIO(text)
    return len(fp.read().split())

In [50]:
def keywords(text):
    fp = StringIO(text)
    word_level_statistics = WordLevelStatistics(corpus_file=fp, percentile_C=90)
    word_level_statistics.compute_spectra()

    lvls_df = pd.DataFrame(word_level_statistics.level_stat_thresholded)
    try:
        lvls_df = lvls_df.sort_values(by='sigma_nor', ascending=False)
    except:
        print(text)
        return ''
    return '|'.join(lvls_df.word[0:5])

In [51]:
df['text length'] = df['text'].map(text_length)
df['keywords'] = df['text'].map(keywords)

佛人 「 シュルリー 」 氏 國 の 衰 に 赴く 徴 を 擧る 條目 左 の 如し 　 杉 亨二 一 運上 を 多分 に 取り立る 事 一 〆賣 商買 を なす 事 但し の 〆賣 商買 を 最も 害 あり と す 一 交易 工作 及 農業 を 怠ら しむる 事 一 運上 を 取り立る に 多く 雜費 を 掛くる 事 一 入ら ぬ 役人 の 多き 事 一 國費 の 多き 事 一 裁判 の ひま とる 事 并 不 相當 の 取計ひ を なす 事 一 怠惰 に し て 勉強 せ ざる 事 一 奢侈 なる 事 一 無益 に 費す 事 一 教方 亂れ て 守り なき 事 一 數 貨幣 を 變る 事 一 數 戰爭 を なす 事 一 政事 の 勝手 次第 なる 事 一 知識 を 弘むる を 忌み嫌ふ 事 一 仕 損 を 打ち て ざる 事 　 右 等 の 類 なり シュルリー 」 は 千 六百 年間 の 人 なり 甞て 佛國 の 宰相 と 爲り 大亂 の 餘 を 承け 其 國 を 治め たり 抑 シュルリー 」 の 此 説 を 發せ し より 當 英蘭 等 の 如き 皆 之 に 從 て を 爲せ り 今 我 邦人 の 此 條 を 見る もの 知ら ず 其 以 て 陳腐 と 爲す や 否 や を 故 に 姑く 録し て 之 を 我 社 の 日誌 に 載す


In [52]:
mask = df.keywords.str.contains('スパルタ')
df[mask]

,index,year,issue,title,author,text,text length,keywords
129,130,1875,34,想像鎖国説（明治八年三月十六日演説）,杉亨二,○ 想像 鎖國 説 明治 八 年 三 月 十 六 日 演説 杉 亨二 世間 に は 常 ...,1753,スパルタ|金銀|自由|か|出來


In [53]:
df.iloc[55].text

'○ 想像 論 \u3000 津田 眞道 想像 は 瞑目 思想 の 間 吾人 覯見 する 所 の 形象 事歴 に し て 頗る 蜃氣 樓 と 相 類似 す 夫の 蜃氣 樓 や 空中 實 に 樓閣 ある に あら ず 而して 歴々 其 形象 を 現す 氣鏡 中 の 射影 なり 吾人 の 心裏 絶 て 天地 萬物 人間 百般 の 事蹟 あ に あら ず 然れ 瞑目 思想 の 間 覯見 す べから ざる 者 なし 心中 の 射影 なり 其 邦訓 おもかげ 想影 の 謂ひ なり 世人 或は 想像 を て おもひや と 云ふ 誤 なり おもひやり は 古語 に 於 て は 排悶 の 義 に し て 俗語 に て は 恕 の 意 なり 想像 記臆 と 異 なり 記臆 は 曾 經歴 し たる ヿ を 記し て 忘れ ざる なり は 未だ 曾て 經歴 せ ざる 所 の 事 を 新 に 結搆 造營 する なり 但し 曾て し たる ヿ に 多少 關係 せ ざる は なし 故 に 記臆 と 關係 尤 親密 なる の 和漢 の 小説 演劇 等 大抵 作者 の 想像 なり 歐洲 各國 に 於 て 尤 有名 なる 詩 の 作詩 も 概する に 皆 想像 なり 各國 太古 の 事蹟 古老 の 相傳 する 所 は 記臆 と 想像 と 相 混淆 す 高天原 黄 龍宮 等 の 事 及び 天竺 埃及 希臘 羅馬 等 の 古傳 皆 荒唐 不經 信 に 足 ず 是 古人 の 想像 なれ ば なり 但 仲尼 書 を 刪り て 斷然 唐虞 以前 の 事 を 採ら ず 豈 其 實事 に あら ず 教 と する に 足ら ざる が 爲 歟 此 叟 眼識 高し と 謂ふ べし 釋迦 耶蘇 倶 に 天堂 地獄 を 説く 其 想像 偶合 する なり 達磨 者流 一撃 に 之 を 看破 す 其 見 亦 高し と 謂ふ べし 然ら ば 則 孔子 達磨 は 絶 て 想 を 用ひ ず 豈 老子 の 所謂 至人 夢 なき 者 の 類 歟 曰く 孔子 達磨 は 大に 想 力 を 用ふる 所 の 人 と す 孔子 の 易 を 説く 達磨 の 坐禪 觀法 皆 一種 の 大 想像 なり 但 其 想像 する 所 稍 高く 稍 古人 と 異 なる のみ 後世 記誦 詞章 五 性理 の 學 精粗 淺深 の 別 あり と 雖 大抵 想像 に あ

In [54]:
mask = df.issue == str(13)
df[mask]

,index,year,issue,title,author,text,text length,keywords
54,55,1874,13,米国政教（三）,加藤弘之,○ 米國 政教 第 六 號 の 加藤 弘之 譯 第 四 章 神道 猶 教法 と 云ふ...,1222,數|或は|論|教會|禮式
55,56,1874,13,想像論,津田真道,○ 想像 論 津田 眞道 想像 は 瞑目 思想 の 間 吾人 覯見 する 所 の 形象 ...,841,人|し|せ|ず|と
56,57,1874,13,民選議院を立るには先政体を定むべきの疑問,阪谷素,○ 民選 議院 を 立る に は 先 政體 を 定む べき の 疑問 阪谷 素 民選 議...,2025,小子|べし|或は|も|益


In [55]:
from ipywidgets import HTML, Image, Layout, Button, Label
from ipywidgets import HBox, VBox, Box

In [56]:
import plotly.graph_objs as go

In [57]:
no_terms = 30
word_list = list(lvls_df['word'].head(no_terms))
positions = [word_level_statistics.word_pos[word] for word in word_list]
keywords_in_context = [' '.join(word_level_statistics.tokens[n-2:n+3]) for n,w in enumerate(word_level_statistics.tokens)]

word_list.reverse()
positions.reverse()

fig1 = go.FigureWidget()
fig1.layout.hovermode = 'closest'
for w, p in zip(word_list, positions):
    scatter = fig1.add_scatter(x=p, y=[w]*len(p))
    scatter.mode = 'markers'
    scatter.marker.symbol = 'line-ns-open'
    scatter.marker.color = 'grey'
    scatter.name = w
    scatter.hovertext = [keywords_in_context[n] for n in p]
    scatter.hoverinfo = 'text'

ticklabels = []
for n in range(1,55):
    if n%2 == 0:
        ticklabels.append(str(n))
    else:
        ticklabels.append('')

layout = go.Layout(
    title='Word Distributions for Top {} Significant Terms'.format(no_terms),
    showlegend=False,
    autosize=True,
#     width=1000,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
#     paper_bgcolor='#7f7f7f',
#     plot_bgcolor='#c7c7c7',
    xaxis=dict(
        title=None,
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        showticklabels=True,
#         ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        tickvals=chapter_boundaries,
        ticktext=ticklabels,
        automargin=True,
        showgrid=True,
        zeroline=False,
        showline=False,
    ),
    yaxis=dict(
        title=None,
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        showticklabels=True,
        automargin=True,
        tickangle=0,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        tickvals=word_list,
        showgrid=True,
        zeroline=False,
        showline=False,
    )
)

fig1.layout = layout

In [58]:
fig1

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': [乳母子 だ 侍従 と いう, 言う て 侍従 が いつも, 入れ…

In [59]:
lvls_df.head(100)

,C,count,sigma_nor,vocab_index,word,reading form
107,52.445577,129,5.420635,386,右近,ウコン
205,19.970726,31,4.089600,1326,常陸介,ヒタチノスケ
219,16.028669,24,3.714686,1573,小君,コギミ
129,27.476590,95,3.657707,517,僧都,ソウズ
218,15.713136,25,3.621541,1552,猫,ネコ
208,16.371923,29,3.588781,1376,少納言,ショウナゴン
196,17.070165,37,3.458995,1144,介,スケ
202,15.952786,33,3.400375,1265,斎宮,サイグウ
230,13.077373,19,3.385558,1884,一条,イチジョウ
180,18.556395,50,3.367266,909,効,カイ


## Word Clusters

In [61]:
import pymagnitude
import hdbscan
import numpy as np
from collections import Counter
try:
    import umap
    print("Using: umap")
except ImportError:
    import bhtsne

Using: umap


In [ ]:
from src.models import enrich_significant_terms, topic_exemplars, display_topics, topic_order_index, hdbscan_parameter_search, enumerate_exemplars
from IPython.core.display import display, HTML

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s %(message)s',"%b-%d-%Y %H:%M:%S")
logger.handlers[0].setFormatter(formatter)
logging.getLogger('joblib').setLevel(logging.ERROR)

In [ ]:
# background_model = '../data/external/wiki-news-300d-1M.magnitude'
background_model = '/Users/ray/data/models/cc.ja.300.magnitude'
background_vectors = pymagnitude.Magnitude(background_model)

In [ ]:
# local_vectors = '../models/hobbit/wordvectors_rare15_spl_window5_bag_hash0_dim200_sqrt_cca_pseudo0_ce0P75_se0.magnitude'
# local_vectors = pymagnitude.Magnitude(local_vectors)

In [ ]:
# vectors = pymagnitude.Magnitude(local_vectors, background_vectors)
# vectors = local_vectors
vectors = background_vectors

In [ ]:
significant_terms = list(lvls_df['word'])
significant_vectors = vectors.query(significant_terms)

In [ ]:
try:
    fit = umap.UMAP(n_neighbors=15, n_components=10, metric='euclidean')
    vec_10d = fit.fit_transform(significant_vectors)
    fit = umap.UMAP(n_neighbors=15, n_components=2, metric='euclidean')
    vec_2d = fit.fit_transform(vec_10d)
except Exception as ex:
    logging.error("Trying bhtsne. Got exception {}".format(ex))
    vec_2d = bhtsne.tsne(np.asfarray(significant_vectors, dtype='float64' ),dimensions=2)

In [ ]:
significant_terms_enriched = enrich_significant_terms(lvls_df, vec_10d, vec_2d, 'leaf')
exemplar_scores, hovers = topic_exemplars(significant_terms_enriched)
summary = pd.DataFrame([h.split(':') for h in hovers], columns=['topic', 'terms'])

In [ ]:
len(significant_terms_enriched[significant_terms_enriched['topic']==-1])

In [ ]:
mask = (lvls_df.word == '器量')
lvls_df[mask]

In [ ]:
def jisho_lookup(word):
    # word = '器量'
    response = requests.get('https://jisho.org/api/v1/search/words?keyword={}'.format(word))
    word_def = response.json()
    # print(word_def)
    jisho_definition = ''
    try:
        jisho_definition = ' | '.join(word_def['data'][0]['senses'][0]['english_definitions'])
    except Exception as ex:
        pass
#         print(word, ex)
    return jisho_definition

In [ ]:
lvls_df['definition'] = list(map(jisho_lookup, lvls_df['word']))

In [ ]:
word_topic_map = dict(lvls_df[['word', 'topic']].values)

In [ ]:
word = 'やっかい'
neighbors = [(word, 1.0)]
neighbors.extend(background_vectors.most_similar_approx(word, topn=15))
for w,s in neighbors:
    t = '_'
    if w in word_topic_map:
        t = word_topic_map[w]
    print(w, t, s, jisho_lookup(w))

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, algorithm='ball_tree').fit(vec_10d)
distances, indices = nbrs.kneighbors(vec_10d)

In [ ]:
index_no = 5
for n, word_num in enumerate(indices[index_no]):
    if distances[index_no][n] < 0.6:
        w = significant_terms[word_num]
        t = '_'
        if w in word_topic_map:
            t = word_topic_map[w]
        print(w, t, distances[index_no][n], jisho_lookup(w))

In [ ]:
topic_no = 7
mask = (lvls_df.topic == topic_no)
lvls_df[mask]

In [ ]:
topics, top_columns = display_topics(significant_terms_enriched, n_rows=20, n_cols=35)
topics = topics.fillna('')
print('{} topics'.format(significant_terms_enriched['topic'].max()))
display(HTML(topics.to_html(index=False)))

In [ ]:
word_list = list(lvls_df[mask]['word'])
positions = [word_level_statistics.word_pos[word] for word in word_list]
keywords_in_context = [' '.join(word_level_statistics.tokens[n-2:n+3]) for n,w in enumerate(word_level_statistics.tokens)]

word_list.reverse()
positions.reverse()

fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
for w, p in zip(word_list, positions):
    scatter = fig.add_scatter(x=p, y=[w]*len(p))
    scatter.mode = 'markers'
    scatter.marker.symbol = 'line-ns-open'
    scatter.marker.color = 'grey'
    scatter.name = w
    scatter.hovertext = [keywords_in_context[n] for n in p]
    scatter.hoverinfo = 'text'

ticklabels = []
for n in range(1,55):
    if n%2 == 0:
        ticklabels.append(str(n))
    else:
        ticklabels.append('')

layout = go.Layout(
    title='Word Distributions for Topic {}'.format(topic_no),
    showlegend=False,
    autosize=True,
#     width=1000,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
#     paper_bgcolor='#7f7f7f',
#     plot_bgcolor='#c7c7c7',
    xaxis=dict(
        title=None,
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        showticklabels=True,
#         ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        tickvals=chapter_boundaries,
        ticktext=ticklabels,
        automargin=True,
        showgrid=True,
        zeroline=False,
        showline=False,
    ),
    yaxis=dict(
        title=None,
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        showticklabels=True,
        automargin=True,
        tickangle=0,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        tickvals=word_list,
        showgrid=True,
        zeroline=False,
        showline=False,
    )
)

fig.layout = layout

In [ ]:
fig

In [ ]:
lvls_df[mask]